
https://github.com/krocki/dnc/blob/master/rnn-numpy.py    
https://blog.csdn.net/tudaodiaozhale/article/details/80464060
    
    
http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-2-implementing-a-language-model-rnn-with-python-numpy-and-theano/     
https://github.com/dennybritz/rnn-tutorial-rnnlm/blob/master/RNNLM.ipynb
    



使用numpy等基本库实现RNN

# 公式
## Notation
输入序列：$x$   
输出序列：$y$   
输入，输出序列中的某个点: $ x^{<t>}, y^{<t>} $  
输入，输出序列的长度: $ T_x^{(i)}, T_y^{(i)} $， 其中i表示第i个样本


![rnn](rnn.png)



## forward
$$
\begin{align}
a^{<t>} =& tanh( w_{aa}a^{<t-1>} + w_{ax}x^{<t>} + b_a )  \\
\\
\hat y^{<t>} =& softmax(w_{ya}^{<t>} a^{<t>} + b_y) \\
\end{align}
$$


## loss
$$
\begin{align}
E^{<t>} = & - \sum_i y_i^{<t>} ln \hat y_i^{<t>} \\
E(y, \hat y) =& - \sum_t E^{<t>} 
\end{align}
$$

## backpropagation

### 对$w_{ya}, b_y$求导

为了方便下面的推导，细化一下上面的forward公式。同时为了简化公式暂时省略$^{<t>}$
$$
\begin{align}
\\
z =& w_{ya} a + b_y \\
z_i =& \sum_j w_{ya\_ij} a_j + b_{y\_i}  \\
\\
\hat y_i =& \frac{ e^{z_i} }{ \sum_j e^{z_j} }  \\
\end{align}
$$

损失函数：
$$
\begin{align}
E =& -\sum_i y_i ln \hat y_i  \\
  =& -\sum_i y_i ( z_i - ln \sum_j e^{z_j}  ) \\
\end{align}
$$

求导
$$
\begin{align}
\frac{ \partial E }{ \partial w_{ya\_ij} } =& \sum_k \frac{ \partial E }{ \partial z_k }
                        \frac{ \partial z_k }{ \partial w_{ya\_ij}}   \\
                                           =& \frac{ \partial E }{ \partial z_i }
                        \frac{ \partial z_i }{ \partial w_{ya\_ij}}                        
\\
\\
\frac{ \partial E }{ \partial z_i } =& \frac{ \partial ( -\sum_{k \neq i} y_k ( z_k - ln \sum_j e^{z_j}  ) - y_i ( z_i - ln \sum_j e^{z_j}  ) ) }{ \partial z_i }  \\
                                    =& -\sum_{k \neq i} y_k (0 - \frac{e^{z_i}}{ \sum_j e^{z_j} } ) - y_i (1 - \frac{e^{z_i} }{ \sum_j e^{z_j} } )  \\
                                    =& \sum_{k \neq i} y_k \frac{e^{z_i}}{ \sum_j e^{z_j} } - y_i + \frac{e^{z_i}}{ \sum_j e^{z_j}}  \\
                                    =& \sum_k y_k \frac{e^{z_i}}{ \sum_j e^{z_j}} - y_i \\
                                    =& \frac{e^{z_i}}{ \sum_j e^{z_j}} - y_i \\
                                    =& \hat y_i - y_i \\
\\
\\
\frac{ \partial z_i }{ \partial w_{ya\_ij}} =& a_j\\
\end{align}
$$

补上$^{<t>}$，得到
$$
\begin{align}
\frac{ \partial E^{<t>} }{ \partial z_i^{<t>} } =&  \hat y_i^{<t>} - y_i^{<t>} \\
\\
\frac{ \partial E^{<t>} }{ \partial w_{ya\_ij} } =& a_j^{<t>}( \hat y_i^{<t>} - y_i^{<t>} )  \\
\\
\frac{ \partial E^{<t>} }{ \partial b_{y\_i} } =& \hat y_i^{<t>} - y_i^{<t>} 
\end{align}
$$



### 对$w_{aa}, w_{ax}, b_a$求导
细化前向公式
$$
\begin{align}
s_i^{<t>} =& \sum_j w_{aa\_ij}a_j^{<t-1>} + \sum_j w_{ax\_ij}x_j^{<t>} + b_{a\_i} \\
a_i^{<t>} =& tanh( s_i^{<t>} )  \\
\\
z_i^{<t>} =& \sum_j w_{ya\_ij} a_j^{<t>} + b_{y\_i}  \\
\end{align}
$$

对$w_{aa}$求导  
在计算$\frac{\partial E^{<t>}}{\partial w_{aa\_ij}}$时需要考虑所有的$s_i^{<t'>}, 0<t' \le t $
$$
\begin{align}
\frac{ \partial E^{<t>} }{ \partial w_{aa\_ij} } =& \sum_{t'} \frac{\partial E^{<t>}}{\partial s_i^{<t'>}}
                                                              \frac{\partial s_i^{<t'>}}{\partial w_{aa\_ij}}
\end{align}
$$

其中第二项$\frac{\partial s_i^{<t'>}}{\partial w_{aa\_ij}}=a_j^{<t'-1>}$。    
第一项计算比较复杂，下面采用递归的方式计算。当$0<t'<t$时：
$$
\begin{align}
\frac{\partial E^{<t>}}{\partial s_i^{<t'>}} =& \sum_k \frac{\partial E^{<t>}}{\partial s_k^{<t'+1>}}
                                                       \frac{\partial s_k^{<t'+1>}}{\partial s_i^{<t'>}} \\
=& \sum_k \sum_m \frac{\partial E^{<t>}}{\partial s_k^{<t'+1>}}
                 \frac{\partial s_k^{<t'+1>}}{\partial a_m^{<t'>}}
                 \frac{\partial a_m^{<t'>}}{\partial s_i^{<t'>}}\\
=& \sum_k \sum_m \frac{\partial E^{<t>}}{\partial s_k^{<t'+1>}}
                w_{aa_km} \delta_{mi} ( 1-(a_i^{<t'>})^2 ) \\
=& \sum_k \frac{\partial E^{<t>}}{\partial s_k^{<t'+1>}}
                w_{aa_ki}( 1-(a_i^{<t'>})^2 ) \\
=& \sum_k w_{aa\_ik}^T  \frac{\partial E^{<t>}}{\partial s_k^{<t'+1>}} ( 1-(a_i^{<t'>})^2 ) 
\end{align}
$$

递推公式的初始值:
$$
\begin{align}
\frac{\partial E^{<t>}}{ \partial s_i^{<t>}} =& \sum_m \sum_n \frac{ \partial E^{<t>} }{ \partial z_m^{<t>} } 
                                                       \frac{ \partial z_m^{<t>}}{ \partial a_n^{<t>} }
                                                       \frac{ \partial a_n^{<t>}}{ \partial s_i^{<t>} } \\
=& \sum_m \sum_n (\hat y_i^{<t>} - y_i^{<t>}) w_{ya\_mn} \delta_{ni} ( 1 - (a_i^{<t>})^2 )\\ 
=& \sum_m (\hat y_i^{<t>} - y_i^{<t>}) w_{ya\_mi} ( 1 - (a_i^{<t>})^2 )\\ 
=& \sum_m w_{ya\_im}^T(\hat y_i^{<t>} - y_i^{<t>})( 1 - (a_i^{<t>})^2 )\\
\end{align}
$$

在算好$\frac{\partial E^{<t>}}{\partial s_i^{<t>}}$后，就比较容易得到对$w_{ax}, b_a$的导数
$$
\begin{align}
\frac{\partial E^{<t>}}{\partial w_{ax\_ij}} =& \sum_{t'} \frac{\partial E^{<t>}}{\partial s_i^{<t'>}}
                                                              \frac{\partial s_i^{<t'>}}{\partial w_{ax\_ij}} \\
=& \sum_{t'} \frac{\partial E^{<t>}}{\partial s_i^{<t'>}} x_j^{<t'>}     \\                                       
\\
\frac{\partial E^{<t>}}{\partial b_{a\_i}} =& \sum_{t'} \frac{\partial E^{<t>}}{\partial s_i^{<t>}}
                                                        \frac{\partial s_i^{<t>}}{\partial b_{a\_i}} \\
=& \sum_{t'} \frac{\partial E^{<t>}}{\partial s_i^{<t'>}}    \\    
\end{align}
$$

### 矩阵形式   

对$w_{ya}, b_y$的导数
$$
\begin{align}
\frac{ \partial E^{<t>} }{ \partial w_{ya} } =& (\hat y^{<t>} - y^{<t>} ) \otimes a^{<t>}   \\
\\
\frac{ \partial E^{<t>} }{ \partial b_y}  =& \hat y^{<t>} - y^{<t>} 
\end{align}
$$

$\frac{\partial E^{<t>}}{\partial s^{<t>}}$的递推公式：
$$
\begin{align}
\frac{\partial E^{<t>}}{ \partial s^{<t>}} =& w_{ya}^T(\hat y^{<t>} - y^{<t>}) * ( 1 - (a^{<t>})^2 )\\
\\
\frac{\partial E^{<t>}}{\partial s^{<t'>}} =& w_{aa}^T \frac{\partial E^{<t>}}{\partial s^{<t'+1>}} * ( 1-(a^{<t'>})^2 ) 
\end{align}
$$

对$w_{aa}, w_{ax}, b_a$的导数
$$
\begin{align}
\frac{\partial E^{<t>}}{\partial w_{aa}} =& \sum_{t'} \frac{\partial E^{<t>}}{\partial s^{<t'>}} \otimes a^{<t'-1>}     \\                   
\frac{\partial E^{<t>}}{\partial w_{ax}} =& \sum_{t'} \frac{\partial E^{<t>}}{\partial s^{<t'>}} \otimes x^{<t'>}
\\
\frac{\partial E^{<t>}}{\partial b_a} =& \sum_{t'} \frac{\partial E^{<t>}}{\partial s^{<t'>}}
\end{align}
$$

In [30]:
import numpy as np
import operator
import logging

In [31]:
def softmax(x):
    e_x = np.exp( x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [32]:
class RNN_NP(object):
    def __init__(self, hid_dim, inp_dim, out_dim):
        self.w_ax = np.random.uniform(-np.sqrt(1.0/inp_dim), np.sqrt(1.0/inp_dim), (hid_dim, inp_dim))
        self.w_aa = np.random.uniform(-np.sqrt(1.0/hid_dim), np.sqrt(1.0/hid_dim), (hid_dim, hid_dim))
        self.w_ya = np.random.uniform(-np.sqrt(1.0/hid_dim), np.sqrt(1.0/hid_dim), (out_dim, hid_dim))
        self.b_a = np.zeros( (hid_dim,), dtype=float )
        self.b_y = np.zeros( (out_dim,), dtype=float )
        self.inp_dim = inp_dim
        self.hid_dim = hid_dim
        self.out_dim = out_dim
        
    def forward(self, x):
        T = len(x)
        a = np.zeros( (T + 1, self.hid_dim), dtype=float )
      
        w_ax = self.w_ax
        w_aa = self.w_aa
        w_ya = self.w_ya
        b_a = self.b_a
        b_y = self.b_y
        y_hat = np.zeros( (T, self.out_dim), dtype=float )
        for t in range(T):
            a[t] = np.tanh(np.dot(w_aa, a[t-1]) + np.dot(w_ax, x[t]) + b_a)
            z = np.dot(w_ya, a[t]) + b_y
            y_hat[t] = softmax(z)
            
        return [y_hat, a]
            
    def predict(self, x):
        y_hat, _ = self.forward(x)
        return np.argmax(y_hat, axis=1)
    
    def calculate_total_loss(self, xs, ylabels):
        L = 0
        for i in np.arange(len(ylabels)):
            y_hat, a = self.forward(xs[i])
            correct_word_predictions = y_hat[np.arange(len(ylabels[i])), ylabels[i]]
            L += -1 * np.sum(np.log(correct_word_predictions))
        return L

    def calculate_loss(self, xs, ylables):
        N = np.sum((len(y) for y in ylables))
        return self.calculate_total_loss(xs,ylables)/N
    
    def backpropagation(self, x, ylabel):
        T = len(ylabel)
        y_hat, a = self.forward(x)
        dw_ax = np.zeros_like(self.w_ax)
        dw_aa = np.zeros_like(self.w_aa)
        dw_ya = np.zeros_like(self.w_ya)
        db_a = np.zeros_like(self.b_a)
        db_y = np.zeros_like(self.b_y)
        
        delta_y = y_hat
        delta_y[np.arange(T), ylabel] -= 1.0
        for t in np.arange(T)[::-1]:
            dw_ya += np.outer( delta_y[t], a[t] )
            db_y += delta_y[t]
            
            delta_t = self.w_ya.T.dot( delta_y[t] ) * ( 1 - (a[t] ** 2) )

            for _t in np.arange(t+1)[::-1]:
                dw_aa += np.outer(delta_t, a[_t-1])
                dw_ax += np.outer(delta_t, x[_t])
                db_a += delta_t
                delta_t = self.w_aa.T.dot(delta_t) * ( 1 - a[_t-1]**2 )
                
        return [dw_aa, dw_ax, db_a, dw_ya, db_y]
    
    
    def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
        bptt_gradients = self.backpropagation(x, y)
   
        param_names = ['w_aa', 'w_ax', 'b_a', 'w_ya', 'b_y']
        for pidx, pname in enumerate(param_names):
            param = operator.attrgetter(pname)(self)
            logging.info("Performing gradient check for parameter %s with shape:%s", pname, param.shape)
            
            it = np.nditer(param, flags=['multi_index'], op_flags=['readwrite'])
            while not it.finished:
                ix = it.multi_index
                org_value = param[ix]
                # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
                param[ix] = org_value + h
                gradplus = self.calculate_total_loss([x],[y])
                param[ix] = org_value - h
                gradminus = self.calculate_total_loss([x],[y])
                estimated_gradient = (gradplus - gradminus)/(2*h)
                param[ix] = org_value
                
                # The gradient for this parameter calculated using backpropagation
                backprop_gradient = bptt_gradients[pidx][ix]
         
                if np.abs(backprop_gradient - estimated_gradient) > error_threshold * (
                    np.abs(backprop_gradient) + np.abs(estimated_gradient) ):
                    logging.info("Gradient Check ERROR: parameter=%s ix=%s", pname, ix)
                    logging.info("+h Loss: %f", gradplus)
                    logging.info("-h Loss: %f", gradminus)
                    logging.info("Estimated_gradient: %f", estimated_gradient)
                    logging.info("Backpropagation gradient: %f", backprop_gradient)
                    logging.info("Relative Error: %f", relative_error)
                     
                it.iternext()
            logging.info("Gradient check for parameter %s passed.", pname)
            
            

In [33]:
def encoder_onehot(x,n):
    e = np.zeros( (n,), dtype=float )
    e[x] = 1
    return e

def print_log(fmt, *params):
    print(fmt % (params))
logging.info = print_log
    
def test_gradient_check():
    np.random.seed(0)
    model = RNN_NP(10, 5, 5)
    x = [ encoder_onehot(i, 5) for i in [0,1,2,3] ]
    #import pdb; pdb.set_trace()
    model.gradient_check(x, [1,2,3,4])
test_gradient_check()

Performing gradient check for parameter w_aa with shape:(10, 10)
Gradient check for parameter w_aa passed.
Performing gradient check for parameter w_ax with shape:(10, 5)
Gradient check for parameter w_ax passed.
Performing gradient check for parameter b_a with shape:(10,)
Gradient check for parameter b_a passed.
Performing gradient check for parameter w_ya with shape:(5, 10)
Gradient check for parameter w_ya passed.
Performing gradient check for parameter b_y with shape:(5,)
Gradient check for parameter b_y passed.
